In [1]:
import socket
import pickle
from boltons import socketutils
from PIL import ImageGrab
import os

server = "localhost"
port = 1234

# screenshoot func
def getScreenShot(buff):
    print('REQUEST SCREENSHOT')
    buff.send('screenshot!F!'.encode())

    dump_size = int(buff.recv_until(b'!!').decode())
    dump = buff.recv_size(dump_size)
    print(f'\tReceived dump data, size: {dump_size}')
    
    return pickle.loads(dump)

# close connection
def getCloseConection(buff):
    print('REQUEST CLOSE')
    buff.send('close!F!'.encode())
    buff.close()

c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
c.connect((server, port))

buff = socketutils.BufferedSocket(c, None)

In [3]:
#test screenshot
img = getScreenShot(buff)
img.show()

REQUEST SCREENSHOT
	Received dump data, size: 6220874
REQUEST CLOSE


In [ ]:
getCloseConection(buff)